#### Importing Required Libraries

In [24]:
import numpy as np
from scipy import sparse
import plotly.graph_objects as go
import plotly.offline as po
from plotly.offline import iplot
import time as t

#### Generating Graph data

In [25]:
s = set() # Set used to hold the nodes in the dataset, its length is the number of nodes in the dataset.
def getGraphData(file):
    '''This function returns a dictionary of edges between the graph nodes read from the file i.e. the adjacency 
    list of the graph data.'''
    graphEdges = dict() # Dictionary to store graph edges
    with open(file) as f: # open the file containing the graph data.
        # Read all lines from the file, a line contains tab seperated source node and dest node.
        lines = f.readline()
        while lines:
            line = [int(node) for node in lines.strip().split('\t')] # get a single line from lines
            # line[0] is the source node, line[1] is the destination node.
            # Storing the edge between line[0] and line[1] in graphEdges dictionary.
            # Adding the unique nodes in set s.
            if graphEdges.get(line[0]):
                s.add(line[1])
                graphEdges[line[0]].append(line[1])
            else:
                graphEdges[line[0]] = [line[1]]
                s.add(line[0])
                s.add(line[1])
            lines = f.readline()
    # len(s) is number of nodes in the data
    return graphEdges, len(s)

In [26]:
# Calling getGraphData() to get the adjacency list of the dataset.
graphEdges, numberOfNodes = getGraphData('/Users/vedantgupta/Desktop/Project/web-Google.txt')

In [27]:
print("Number of nodes in our dataset", numberOfNodes)

Number of nodes in our dataset 875713


In [28]:
# Highest index node in our dataset
highest_index = sorted(graphEdges.keys(), reverse=True)[:1][0]+1

In [29]:
print("Highest index node in our dataset",highest_index) # We will pass this as the size of the matrix

Highest index node in our dataset 916428


#### Unweighted Page Rank Algorithms

In [30]:
def createSparseMatrixFromGraph(graph, size):
    '''This function creates a sparse matrix from the adjacency list.'''
    row = [] # the row index of the matrix
    col = [] # the col index of the matrix
    data = [] # The value to be stored at the row index and col index
    for node in graph.keys(): # iterate over all the nodes 
        nodeVal = graph.get(node) # the list of nodes to which the edge goes to from nodeKey
        # We assign equal node values to all the nodes which is 1/number of nodes the edges goes to
        Count = len(nodeVal)
        eachNodeVal = 1 / Count
        # for handling duplicates, for duplicates we assign more weights
        countDict = dict()
        for val in nodeVal:
            if countDict.get(val):
                countDict[val] = countDict.get(val) + 1
            else:
                countDict[val] = 1
        for node_value in list(set(nodeVal)):
            count = countDict.get(node_value) # get the count of the node from val dict
            row.append(node_value) # append key to the row matrix
            col.append(node) # append the node to which the connection goes to in the column matrix
            # multiply with count the decided weight to get the final value.
            data.append(count * float(eachNodeVal))
    # using sparse from scipy to generate the matrix
    matrix = sparse.csr_matrix((data, (row, col)), shape=(size, size))
    return matrix

In [31]:
# Getting the sparse matrix of our data
sparseMatrix = createSparseMatrixFromGraph(graphEdges, highest_index)

In [32]:
# Number of non_zero edges in our matrix
numberOfNonZeroEdges = sparse.csr_matrix.count_nonzero(sparseMatrix)

In [33]:
print('Number of non zero points is ',numberOfNonZeroEdges)

Number of non zero points is  5105039


In [34]:
# Parameters for the various algorithms for page rank
beta = 0.8 # damping factor
tolerance = 10**-10 # tolerance
TOP_K = 7 # Return to the TOP_K related pages
dValueForExtraPolation = [8] # d value for extrapolation for accelarating page rank

In [35]:
def power_iteration(sparse_matrix,beta,tolerance,number_of_nodes):
    # Given, 1 is one vector with nx1 entries of value 1
    one = np.ones((number_of_nodes,1))
    # initialising the r0
    r = 1/ float(number_of_nodes) * one
    iteration = 0 # number of iterations for the algo to get the output.
    while True:
        iteration += 1 # increment the count
        rnew = ((1-beta)/number_of_nodes) * one + beta * sparse_matrix.dot(r) # new r value for all the nodes
        l1Norm = np.linalg.norm(abs(rnew - r), ord=1) # calculate the error, used l1norm
        if l1Norm < tolerance: # If error is less than tolerance break
            break
        r = rnew # return the new r and the number of iterations it required the algo to0 converge.
    return r,iteration

In [36]:
def power_extrapolation(transition_matrix, number_of_nodes, dValueForExtraPolation, beta, tolerance):
    dValues = dValueForExtraPolation # D values to accelarate page rank
    d_0 = dValues[0] # initial d_value
    pageRank = {}  # Dictionary to store page rank
    residual = {} # dictionary to store errors
    for d_val in dValues: # iterate over dvalue array
        page_rank_iteration = []
        initial_pagerank = 1 / number_of_nodes * np.ones((number_of_nodes, 1)) # calculate initial page rank
        iteration = 0 # initial iteration
        r_error = {} # dict to store erorr
        page_rank_iteration.append(initial_pagerank) 
        while True: # loop till error becomes less than tolerance
            iteration += 1 # increment the count
            # calculate the next page rank
            next_pagerank = (1 - beta) / number_of_nodes * np.ones((number_of_nodes, 1)) + beta * transition_matrix.dot(initial_pagerank)
            if iteration == d_val + 2:
                next_pagerank = (next_pagerank - (beta**d_val)*page_rank_iteration[iteration - d_val]) / (1 - beta**d_val)
            error = np.linalg.norm(abs(next_pagerank - initial_pagerank), ord=1) # calculate the error
            r_error[iteration] = error 
            if error < tolerance: # if error becomes less than the tolerance break
                page_rank_iteration.append(next_pagerank)
                break
            page_rank_iteration.append(next_pagerank) 
            initial_pagerank = next_pagerank
        pageRank[d_val] = page_rank_iteration
        residual[d_val] = r_error
    # return the final page rank and the iteration.
    return pageRank.get(d_0)[-1], iteration

In [37]:
def getTopBottomK(topK,r):
    # Top k page ranks with values
    topK_page_Id = r.T[0].argsort()[-topK:][::-1]
    topK_pages = getValuesTuple(topK_page_Id,r)
    print('#################### Top k pages with scores ####################')
    for page_rank in enumerate(topK_pages):
        (index, (page_rank_Val, nodeId)) = page_rank
        print('Page rank ', index+1, 'for page id ',nodeId+1,'with value is - ', page_rank_Val)
    print('\n\n')
    # Bottom K page ranks with values
    bottomK_page_Id = r.T[0].argsort()[:topK]
    bottomK_pages = getValuesTuple(bottomK_page_Id,r)
    print('#################### Bottom k pages with scores ####################')
    for pr in enumerate(bottomK_pages):
        (index, (page_rank_Val, nodeId)) = pr
        print('Page rank ', index+1, 'for page id ',nodeId+1,'with value is - ', page_rank_Val)

In [38]:
def getValuesTuple(indices, r):
    '''Function to get value tuples'''
    return [(r[index][0], index+1) for index in indices]

In [39]:
# Using normal page rank and power iteration
st=t.time()
r,count = power_iteration(sparseMatrix,beta,tolerance,highest_index)
print("Normal page rank with power iteration:")
print('\n')
getTopBottomK(TOP_K,r)
print('\n')
print(t.time()-st)

Normal page rank with power iteration:


#################### Top k pages with scores ####################
Page rank  1 for page id  163077 with value is -  0.0006286998490852598
Page rank  2 for page id  597623 with value is -  0.0006247975197122885
Page rank  3 for page id  537041 with value is -  0.0006126822588027786
Page rank  4 for page id  41911 with value is -  0.000597238903366512
Page rank  5 for page id  384668 with value is -  0.0005158512929324215
Page rank  6 for page id  504142 with value is -  0.0005136825190031836
Page rank  7 for page id  605858 with value is -  0.0005033165721127062



#################### Bottom k pages with scores ####################
Page rank  1 for page id  242137 with value is -  2.1823863958761623e-07
Page rank  2 for page id  555801 with value is -  2.1823863958761623e-07
Page rank  3 for page id  257350 with value is -  2.1823863958761623e-07
Page rank  4 for page id  257352 with value is -  2.1823863958761623e-07
Page rank  5 for page id  2

In [40]:
# Using normal page rank and power extrapolation
st=t.time()
r, count = power_extrapolation(sparseMatrix, highest_index, dValueForExtraPolation, beta, tolerance)
print("Normal page rank with power extrapolation")
print('\n')
getTopBottomK(TOP_K,r)
print('\n')
print(t.time()-st)

Normal page rank with power extrapolation


#################### Top k pages with scores ####################
Page rank  1 for page id  163077 with value is -  0.0006286998488630672
Page rank  2 for page id  597623 with value is -  0.0006247975193672014
Page rank  3 for page id  537041 with value is -  0.0006126822584833776
Page rank  4 for page id  41911 with value is -  0.0005972389029305972
Page rank  5 for page id  384668 with value is -  0.0005158512925573922
Page rank  6 for page id  504142 with value is -  0.00051368251869735
Page rank  7 for page id  605858 with value is -  0.0005033165719936103



#################### Bottom k pages with scores ####################
Page rank  1 for page id  242137 with value is -  2.1823863958761623e-07
Page rank  2 for page id  555801 with value is -  2.1823863958761623e-07
Page rank  3 for page id  257350 with value is -  2.1823863958761623e-07
Page rank  4 for page id  257352 with value is -  2.1823863958761623e-07
Page rank  5 for page id 

#### Weighted

In [41]:
def createSparseWeightedMatrixFromGraph(graph, size):
    row = [] # row index of the matrix
    col = [] # column index of the matrix
    data = [] # The value to be stored at the row index and col index 
    for node in graph.keys(): # iterate over all the nodes
        nodeVal = graph.get(node) # the list of nodes to which the edge goes to from nodeKey
        # We assign equal node values to all the nodes which is 1/number of nodes the edges goes to
        Count = len(nodeVal) 
        eachNodeVal = 1 / Count
        # for handling duplicates, for duplicates we assign more weights
        countDict = dict()
        for val in nodeVal:
            if countDict.get(val):
                countDict[val] = countDict.get(val) + 1
            else:
                countDict[val] = 1
        for node_value in list(set(nodeVal)):
            count = countDict.get(node_value) # get the count of the node from val dict
            row.append(node_value) # append key to the row matrix
            col.append(node) # append the node to which the connection goes to in the column matrix
            data.append(count) # append the count to the data list.
    matrix = sparse.csr_matrix((data, (row, col)), shape=(size, size))
    
    row_sum = matrix.sum(axis=1).T.tolist()[0] # calculate the sum of rows, used in assigning weights
    col_sum = matrix.sum(axis=0).tolist()[0] # calculate the sum of columns, used in assigning weights
    row = [] # row index of the matrix
    col = [] # column index of the matrix
    data = [] # The value to be stored at the row index and col index 
    for node in graph.keys(): # iterate over all the nodes
        nodeVal = graph.get(node) # the list of nodes to which the edge goes to from nodeKey
        Count = len(nodeVal)
        eachNodeVal = 1 / Count
        # for handling duplicates, for duplicates we assign more weights
        countDict = dict()
        for val in nodeVal:
            if countDict.get(val):
                countDict[val] = countDict.get(val) + 1
            else:
                countDict[val] = 1
        #Incoming weight for children node
        sum_In = 0
        for node_value in list(set(nodeVal)):
            sum_In = sum_In + row_sum[node_value]
        #Outgoing weight for children node
        sum_Out = 0
        for node_value in list(set(nodeVal)):
            sum_Out = sum_In + col_sum[node_value]
        # Calculating the incoming and outgoing weight, assigning weight as the product of two.
        for node_value in list(set(nodeVal)):
            node_in = row_sum[node_value]
            incoming_weight = float(node_in/sum_In)
            
            node_out = col_sum[node_value]
            outgoing_weight =float(node_out/sum_Out)
            row.append(node_value)
            col.append(node)
            data.append(incoming_weight*outgoing_weight)
    # return the matrix
    matrix = sparse.csr_matrix((data, (row, col)), shape=(size, size))
    return matrix

In [42]:
sparseMatrixWeighted = createSparseWeightedMatrixFromGraph(graphEdges, highest_index)

In [43]:
# Using weighted page rank and power iteration
st=t.time()
r, count = power_iteration(sparseMatrixWeighted, beta, tolerance, highest_index)
print("Weighted page rank with power iteration")
print('\n')
getTopBottomK(TOP_K,r)
print('\n')
print(t.time()-st)

Weighted page rank with power iteration


#################### Top k pages with scores ####################
Page rank  1 for page id  285816 with value is -  1.764525840104099e-05
Page rank  2 for page id  751386 with value is -  9.970700997967997e-06
Page rank  3 for page id  321968 with value is -  8.871338801203879e-06
Page rank  4 for page id  868760 with value is -  7.194804372269576e-06
Page rank  5 for page id  616401 with value is -  6.867585406157204e-06
Page rank  6 for page id  837837 with value is -  5.709059907071018e-06
Page rank  7 for page id  163077 with value is -  5.698678681885871e-06



#################### Bottom k pages with scores ####################
Page rank  1 for page id  308773 with value is -  2.1823863958761623e-07
Page rank  2 for page id  298902 with value is -  2.1823863958761623e-07
Page rank  3 for page id  298900 with value is -  2.1823863958761623e-07
Page rank  4 for page id  298899 with value is -  2.1823863958761623e-07
Page rank  5 for page id

In [44]:
# Using weighted page rank and power extrapolation
st=t.time()
r, count = power_extrapolation(sparseMatrixWeighted,highest_index, dValueForExtraPolation, beta, tolerance)
print("Weighted page rank with power extrapolation")
print('\n')
getTopBottomK(TOP_K,r)
print('\n')
print(t.time()-st)

Weighted page rank with power extrapolation


#################### Top k pages with scores ####################
Page rank  1 for page id  285816 with value is -  1.7645258401040716e-05
Page rank  2 for page id  751386 with value is -  9.970700997930113e-06
Page rank  3 for page id  321968 with value is -  8.87133880120335e-06
Page rank  4 for page id  868760 with value is -  7.194804372269045e-06
Page rank  5 for page id  616401 with value is -  6.867585406157204e-06
Page rank  6 for page id  837837 with value is -  5.70905990694615e-06
Page rank  7 for page id  163077 with value is -  5.698678681878906e-06



#################### Bottom k pages with scores ####################
Page rank  1 for page id  308773 with value is -  2.1823863958761623e-07
Page rank  2 for page id  298902 with value is -  2.1823863958761623e-07
Page rank  3 for page id  298900 with value is -  2.1823863958761623e-07
Page rank  4 for page id  298899 with value is -  2.1823863958761623e-07
Page rank  5 for page

#### Analysis

In [45]:
beta_arr=np.arange(0,1,0.1) # Range of damping factor
tol_arr=[10**-10,10**-9,10**-8,10**-7,10**-6,10**-5,10**-4,10**-3,10**-2,10**-1] # range of tolerance values

#### Generating the graph

In [46]:
n_iter_1=np.arange(0,1,0.1)
n_iter_2=np.arange(0,1,0.1)
n_iter_3=np.arange(0,1,0.1)
n_iter_4=np.arange(0,1,0.1)

n_iter_1_tol=np.arange(0,1,0.1)
n_iter_2_tol=np.arange(0,1,0.1)
n_iter_3_tol=np.arange(0,1,0.1)
n_iter_4_tol=np.arange(0,1,0.1)


for i in range(len(beta_arr)):
    r,n_iter_1[i] = power_iteration(sparseMatrix,beta_arr[i],tolerance, highest_index)
    r,n_iter_2[i] = power_extrapolation(sparseMatrix, highest_index,dValueForExtraPolation,beta_arr[i],tolerance)
    r,n_iter_3[i] = power_iteration(sparseMatrixWeighted,beta_arr[i],tolerance, highest_index)
    r,n_iter_4[i] = power_extrapolation(sparseMatrixWeighted,highest_index,dValueForExtraPolation,beta_arr[i],tolerance)
    

for i in range(len(tol_arr)):
    r,n_iter_1_tol[i] = power_iteration(sparseMatrix,beta,tol_arr[i], highest_index)
    r,n_iter_2_tol[i] = power_extrapolation(sparseMatrix, highest_index,dValueForExtraPolation,beta,tol_arr[i])
    r,n_iter_3_tol[i] = power_iteration(sparseMatrixWeighted,beta,tol_arr[i], highest_index)
    r,n_iter_4_tol[i] = power_extrapolation(sparseMatrixWeighted,highest_index,dValueForExtraPolation,beta,tol_arr[i])
    
# Create traces
fig = go.Figure()
fig.add_trace(go.Scatter(x=beta_arr, y=n_iter_1,
                    mode='lines+markers',
                    name='PageRank-Power_Iteration '))
fig.add_trace(go.Scatter(x=beta_arr, y=n_iter_2,
                    mode='lines+markers',
                    name='PageRank-Power_Extrapolation'))
fig.add_trace(go.Scatter(x=beta_arr, y=n_iter_3,
                    mode='lines+markers', name='Weighted_PageRank-Power_Iteration'))

fig.add_trace(go.Scatter(x=beta_arr, y=n_iter_4,
                    mode='lines+markers', name='Weighted_PageRank-Power_Extrapolation'))

fig.update_layout(title='No of iterations vs Damping Factors',
                   xaxis_title='Damping Factors',
                   yaxis_title='No of Iterations')

fig.show()

#po.plot(fig, filename = 'Damping_factor.png', auto_open=False)
fig2 = go.Figure()
fig2.add_trace(go.Scatter(x=tol_arr, y=n_iter_1_tol,
                    mode='lines+markers',
                    name='PageRank-Power_Iteration '))
fig2.add_trace(go.Scatter(x=tol_arr, y=n_iter_2_tol,
                    mode='lines+markers',
                    name='PageRank-Power_Extrapolation'))
fig2.add_trace(go.Scatter(x=tol_arr, y=n_iter_3_tol,
                    mode='lines+markers', name='Weighted_PageRank-Power_Iteration'))

fig2.add_trace(go.Scatter(x=tol_arr, y=n_iter_4_tol,
                    mode='lines+markers', name='Weighted_PageRank-Power_Extrapolation'))

fig2.update_layout(title='No of iterations vs Tolerance',
                   xaxis_title='Tolerance',xaxis_type="log",
                   yaxis_title='No of Iterations')

fig2.show()

In [ ]:
/Users/vedantgupta/Documents/Zoom/2021-12-14 16.44.34 vedant gupta's Zoom Meeting/video1059211311.mp4